In [2]:
import pandas as pd
import numpy as np
import sys
import os

from gensim.models import Word2Vec

In [3]:
def get_Xy(df, model, max_word_n):

    def tune_size(i, words):
        words = [model.wv[word] for word in words][:max_word_n]
        if i%100 == 0:
            percent = i/data_n*100
            sys.stdout.write("\r% 5.2f%%"%(percent))
        return np.array(words+placeholder[len(words):])

    data_n = len(df)
    dim_n = len(model.wv[model.wv.index2word[0]])
    placeholder = [np.array([0]*dim_n)]*max_word_n
    
    X = np.array([tune_size(i, words)
                  for i, words in enumerate(df["review"])])
    sys.stdout.write("\r% 5.2f%%\n"%100)
    
    y = np.array([[1, 0] if l == "neg" else [0, 1] for l in df["label"]])
    
    return X, y

In [5]:
for case in ["underscore"]:
    
    if case == "normal":
        max_word_n = 80
    elif case == "underscore":
        max_word_n = 160

    df = pd.read_pickle("./data/%s_df"%case)
    data_n = len(df)

    model = pd.read_pickle("./data/%s_model"%case)

    X, y = get_Xy(df, model, max_word_n)
    print(X.shape, y.shape)

    pd.to_pickle(X[:data_n//2], "./data/%s_te_X"%case)
    pd.to_pickle(X[data_n//2:], "./data/%s_tr_X"%case)

pd.to_pickle(y[:data_n//2], "./data/te_y")
pd.to_pickle(y[data_n//2:], "./data/tr_y")

 100.00%
(50000, 160, 128) (50000, 2)
